First step grabs json data into Python.  Need to split the json objects since python doesn't know how to use multiple json objects.

import json
import string
import pandas as pd
from pandas.io.json import json_normalize


with open('C:/Users/TheDave/Documents/GitHub/Scrapy/luxrent/rent.json') as jsonfile:
    updatedData = jsonfile.read().replace('][', ']<SPLIT>[')
    splitData = updatedData.split('<SPLIT>')
    
    luxrent = [json.loads(bitData) for bitData in splitData]


#luxrent = pd.DataFrame(luxrent)
json_normalize(luxrent['Date'] )

print(luxrent)

Grab Sqlite Database to Python.  Gives information on Table name, column names.

In [7]:
import sqlite3
from sqlite3 import Error
import pandas as pd
from sqlalchemy import create_engine
from IPython.display import display, HTML

#Panda's method of usiing SQLalchemy to grab my SQLite data
engine = create_engine('sqlite:///C:/Users/TheDave/Desktop/LuxData.sqlite')
conn = sqlite3.connect('C:/Users/TheDave/Desktop/LuxData.sqlite')
c = conn.cursor()

#get Table name
res = c.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print ("Table Name = "+ name[0])

#get names of all the columns
c.execute("select * from Lux")
col_names = [description[0] for description in c.description]
print(col_names)

c.close()
conn.close()
    

Table Name = Lux
['id', 'Date', 'Company', 'Building', 'Floorplan', 'City', 'Localaddress', 'Floor', 'Roomnumber', 'Dateavailable', 'Price']


c.execute("select distinct Building from Lux")
print(c.fetchall())

print(c.fetchone())

Goal:

1) To find the vacancy rate per apartment building per day<br>
2) How long rooms are available and how much money AMLI is losing.<br>

Procedure: 

1) Filter information by Building<br>
2) Get list of rooms for each apartment<br>
3) Get Date Range they are each open for<br>
4) Calculate how much money they lost<br>

In [2]:
#This gives the total number of Rooms per apartment complex
buildList = pd.read_sql_query("select distinct building from Lux", engine)


Total = {'TotalRooms':[375,231,220,318,290,279,360,154,175]}
buildTotal = pd.DataFrame(Total)

buildList = pd.concat([buildList,buildTotal],axis=1)
d = 'on 2ND'
buildList['TotalRooms'].loc[buildList['Building']== d].values[0]

#print ('# of buildings = ',len(buildList.Building))

231

In [3]:
#All Dates I have saved
DatesAvailable = pd.read_sql_query("select distinct Date from Lux ", engine)
#DatesAvailable

In [4]:
openRoomsQuery = ("select id, Date, Building, Floorplan, Roomnumber, Dateavailable, Price" 
" from Lux where Dateavailable like 'Now%' ")

openRooms = pd.read_sql_query(openRoomsQuery, engine)
openRooms = openRooms.sort_values(by=['Building','Roomnumber'])
#openRooms


In [10]:
uniqueRooms = pd.unique(openRooms.Roomnumber)
for u in uniqueRooms:
  break  
uniqueRooms[0]

'01-001'

In [6]:
buildingRatio = pd.DataFrame(columns=['Date','Building','Vacancy Rate'])
for d in DatesAvailable['Date']:
    for b in buildList['Building']:
        #print (b)
        #print(openRooms.loc[(openRooms['Date']==d) & (openRooms['Building'] ==b)])
        nRooms = len(openRooms.loc[(openRooms['Date']==d) & (openRooms['Building'] ==b)])
        tRooms = buildList['TotalRooms'].loc[buildList['Building']== b].values[0]
        #buildList.loc[buildList['Building']== b].TotalRooms[0]
        fRooms = nRooms/tRooms *100
        listRooms = [{'Date':d, 'Building':b, 'Vacancy Rate':fRooms}]
        listRooms=pd.DataFrame(listRooms)
        buildingRatio=pd.concat([buildingRatio,listRooms])
    
buildingRatio.groupby('Building').mean()


,Vacancy Rate
Building,
300,6.130310
5350,5.704600
Covered Bridge,6.226460
Downtown,1.879815
Eastside,6.054939
South Shore,4.264407
at Mueller,4.841747
on 2ND,2.964267
on Aldrich,8.445262


Vacancy rating is close to the average in 2018 at 4%.

Use ML to predict next rooms.

Input = Building, Floorplan, Address, Floor, Price
Y = Classification if rented by next month (Y or N)
Synth Data = How Long room open? Price changed?  (Need to check these data points)

In [20]:
roomsPredQuery = ("select Date, Building, Floorplan, Localaddress, Floor," 
                  "Roomnumber, Dateavailable, Price from Lux")
 
roomsPred = pd.read_sql_query(roomsPredQuery, engine)
roomsPred = roomsPred.sort_values(by=['Building','Roomnumber'])

roomsPred.describe()

,Date,Building,Floorplan,Localaddress,Floor,Roomnumber,Dateavailable,Price
count,24461,24461,24461,24461,24461,24461,24461,23427
unique,118,9,111,9,19,579,317,1482
top,04/12/17,on Aldrich,A3a,2401 Aldrich St,2,01-229,Now,"$1,880"
freq,242,4902,815,4902,6435,236,3169,185
